In [6]:
import pandas as pd
import folium
from folium.plugins import MiniMap
from pathlib import Path
import json

# === 1. Baca data dari Google Sheets ===
url = "https://docs.google.com/spreadsheets/d/1iNevuUEhHf16MPlA-WDD1vDxiTRc5A4hGeBHZjh4XPk/export?format=csv"
data = pd.read_csv(url)

data.columns = data.columns.str.strip().str.upper()
data["JENIS"] = data["JENIS"].astype(str).str.strip().fillna("Lainnya")

# === 2. Lokasi awal peta ===
center_lat = data["LINTANG"].mean()
center_lon = data["BUJUR"].mean()

# === 3. Peta utama ===
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=6,
    tiles="OpenStreetMap",
    zoom_control=False
)

# Basemap tambahan
folium.TileLayer("cartodb positron", name="Carto Light").add_to(m)
folium.TileLayer("cartodb dark_matter", name="Carto Dark").add_to(m)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles © Esri",
    name="Esri Satellite"
).add_to(m)

# === 4. Layer Kawasan Konservasi dengan warna berdasarkan field "ZONE" ===
geojson_path = Path("Zonasi-Kawasan-Konservasi.json")
with open(geojson_path, encoding="utf-8") as f:
    kawasan_data = json.load(f)

zone_colors = {
    "Zona Inti": "#FF0000",                  # Merah
    "Zona Pemanfaatan Terbatas": "#96FF96",  # Kuning
    "Zona Lain": "#828282"                   # Hijau (default)
}

def style_by_zone(feature):
    zone = feature["properties"].get("ZONE", "Zona Lain")
    color = zone_colors.get(zone, "#00FF00")
    return {
        "fillColor": color,
        "color": color,
        "weight": 1,
        "fillOpacity": 0.5
    }

folium.GeoJson(
    kawasan_data,
    name="Kawasan Konservasi",
    style_function=style_by_zone,
    tooltip=folium.features.GeoJsonTooltip(
        fields=["NKK", "NZ_KK"],
        aliases=["Kawasan Konservasi:", "Zonasi:"],
        localize=True
    )
).add_to(m)

tooltip_style = """
<style>
.leaflet-tooltip {
    font-size: clamp(13px, 1.2vw, 16px) !important;
    font-weight: 600;
    color: #000;
    background: rgba(255, 255, 255, 0.9);
    border: 1px solid #666;
    border-radius: 6px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.2);
    padding: 6px 10px;
    line-height: 1.4;
}
</style>
"""
m.get_root().html.add_child(folium.Element(tooltip_style))

# === Tambahkan font custom glyph dari GitHub ===
custom_font_css = """
<style>
@font-face {
    font-family: 'BiotaGlyph';
    src: url('https://bkkpn-kupang.github.io/IGTBiotaLautDilindungi/Igtbiotalautdilindungi-Regular.ttf') format('truetype');
}
.glyph-icon {
    font-family: 'BiotaGlyph';
    font-size: 20px;
    color: white;
    text-align: center;
    line-height: 36px;
}
</style>
"""
m.get_root().html.add_child(folium.Element(custom_font_css))

# === 6. Checkbox "Centang Semua" ===
checkall_js = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var interval = setInterval(function() {
        var control = document.querySelector('.leaflet-control-layers-overlays');
        if (control) {
            clearInterval(interval);
            var wrapper = document.createElement('div');
            wrapper.innerHTML = `
                <label style="font-weight:600; display:flex; align-items:center; margin-bottom:4px;">
                    <input type="checkbox" id="toggle-all" style="margin-right:6px;">
                    Filter / Hapus Filter
                </label>
            `;
            control.insertBefore(wrapper, control.firstChild);
            var toggleAll = document.getElementById('toggle-all');
            toggleAll.addEventListener('change', function() {
                var boxes = control.querySelectorAll('input.leaflet-control-layers-selector[type="checkbox"]');
                boxes.forEach(function(box) {
                    if (box.checked !== toggleAll.checked) box.click();
                });
            });
        }
    }, 300);
});
</script>
"""
m.get_root().html.add_child(folium.Element(checkall_js))

# === 7. Marker berdasarkan jenis (dengan fallback otomatis) ===
base_icon_url = "https://bkkpn-kupang.github.io/Imageicon/"

for jenis in data["JENIS"].unique():
    group = folium.FeatureGroup(name=f"{jenis}", show=True).add_to(m)
    subset = data[data["JENIS"] == jenis]

    for _, row in subset.iterrows():
        popup_text = f"""
        <b>KELOMPOK:</b> {row.get('KELOMPOK','')}<br>
        <b>SPESIES:</b> {row.get('SPESIES','')}<br>
        <b>NAMA:</b> {row.get('NAMA_ID','')}<br>
        <b>JUMLAH KEMUNCULAN:</b> {row.get('JUMLAH_TEMU','')}<br>
        <b>DESKRIPSI:</b> {row.get('DESKRIPSI','')}<br>
        <b>TAHUN:</b> {row.get('TAHUN','')}<br>
        <b>DOKUMENTASI:</b> <a href="{row.get('DOKUMENTASI','#')}" target="_blank">Klik di sini</a>
        """

        # Nama file ikon disesuaikan dengan pola
        icon_filename = f"icon{jenis.lower().replace(' ', '').replace('-', '')}.png"
        icon_url = f"{base_icon_url}{icon_filename}"
        icon = folium.CustomIcon(icon_image=icon_url, icon_size=(40, 40))

        folium.Marker(
            location=[row["LINTANG"], row["BUJUR"]],
            popup=folium.Popup(popup_text, max_width=450),
            tooltip=row.get("NAMA_UMUM", ""),
            icon=icon
        ).add_to(group)

icon = folium.CustomIcon( 
    icon_image=icon_url, 
    icon_size=(36, 36), 
    icon_anchor=(18, 18) 
)

popup_style = """
<style>
.leaflet-popup-content {
    font-size: clamp(13px, 1.2vw, 16px);
    line-height: 1.6;
    font-family: 'Segoe UI', Arial, sans-serif;
    color: #1b1b1b;
}

.leaflet-popup-content b {
    color: #0b3d91;
    font-weight: 700;
}

.leaflet-popup-content a {
    color: #007BFF;
    text-decoration: none;
    font-weight: 600;
}

.leaflet-popup-content a:hover {
    text-decoration: underline;
}
</style>
"""
m.get_root().html.add_child(folium.Element(popup_style))

# === 8. LayerControl ===
layer_control = folium.LayerControl(collapsed=False)
layer_control.add_to(m)

# Tambahkan CSS & JS agar layer control otomatis disembunyikan di layar kecil
responsive_layer_control = """
<style>
@media (max-width: 768px) {
    .leaflet-control-layers {
        display: none !important;
    }
    #toggle-layer-btn {
        display: block !important;
    }
}
@media (min-width: 769px) {
    #toggle-layer-btn {
        display: none !important;
    }
}
#toggle-layer-btn {
    position: fixed;
    bottom: 100px;
    right: 15px;
    z-index: 9999;
    background: rgba(11,61,145,0.9);
    color: white;
    border: none;
    border-radius: 8px;
    padding: 8px 12px;
    cursor: pointer;
    font-size: 13px;
    box-shadow: 0 3px 10px rgba(0,0,0,0.3);
}
</style>

<button id="toggle-layer-btn">🗂️ Layer</button>

<script>
// Pastikan tombol baru aktif setelah semua elemen Leaflet sudah siap
window.addEventListener('load', function() {
  const toggleBtn = document.getElementById('toggle-layer-btn');
  const observer = new MutationObserver(() => {
    const control = document.querySelector('.leaflet-control-layers');
    if (control && toggleBtn) {
      toggleBtn.addEventListener('click', () => {
        if (control.style.display === 'none' || control.style.display === '') {
          control.style.display = 'block';
        } else {
          control.style.display = 'none';
        }
      });
      observer.disconnect();
    }
  });
  observer.observe(document.body, { childList: true, subtree: true });
});
</script>
"""
m.get_root().html.add_child(folium.Element(responsive_layer_control))

m.get_root().html.add_child(folium.Element("""
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0">
"""))

# === Tambahkan meta viewport agar tidak auto-scaling di layar besar atau mobile ===
m.get_root().html.add_child(folium.Element("""
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0">
"""))

# === 9. Judul & tombol responsif (dengan ukuran stabil di semua scale display) ===
title_html = """
<div class="info-box">
    <div class="logo-container">
        <img src="https://bkkpn-kupang.github.io/Logo/LogoKKP.png" alt="Logo KKP" class="logo-item">
        <img src="https://bkkpn-kupang.github.io/Logo/LogoBKKPN.png" alt="Logo BKKPN Kupang" class="logo-item">
    </div>

    <div class="title-text">🌊 PETA SEBARAN BIOTA LAUT DILINDUNGI</div>
    <div class="subtitle-text">Balai Pengelolaan Kelautan Kupang</div>

    <div class="desc-text">(Last Update 13 Februari 2026)</div>
    <div class="desc-text">Klik tombol di bawah untuk melihat grafik monitoring dan dashboard:</div>

    <div class="button-container">
        <a href="https://lookerstudio.google.com/u/0/reporting/faefd501-627c-47bf-b741-b25ebb73af8c/page/p_tu61f29uwd"
           target="_blank" class="btn blue">📊 Lihat Grafik Monitoring</a>
        <a href="https://sites.google.com/view/biota-laut-dilindungi/home"
           target="_blank" class="btn green">🌐 Buka Dashboard SI MOLI</a>
    </div>
</div>

<style>
.info-box {
    position: fixed;
    top: 2vh;
    left: 2vw;
    background-color: rgba(255, 255, 255, 0.95);
    padding: 1.5em;
    border-radius: 1em;
    font-family: 'Segoe UI', Arial, sans-serif;
    box-shadow: 0 0.4em 1em rgba(0,0,0,0.25);
    z-index: 9999;
    max-width: 28vw;
    text-align: center;
    backdrop-filter: blur(6px);
    transform: scale(1);
    transform-origin: top left;
}

.logo-container {
    display: flex;
    justify-content: center;
    align-items: center;
    gap: 1em;
    margin-bottom: 0.8em;
}

.logo-item {
    height: clamp(34px, 2.8vw, 45px);
    object-fit: contain;
}

.title-text {
    font-size: clamp(16px, 1.6vw, 22px);
    font-weight: 800;
    color: #0b3d91;
    margin-bottom: 0.3em;
}

.subtitle-text {
    font-size: clamp(12px, 1.1vw, 14px);
    font-weight: 600;
    color: #004080;
    margin-bottom: 0.6em;
}

.desc-text {
    font-size: clamp(13px, 1vw, 15px);
    color: #2c3e50;
    line-height: 1.4;
}

.button-container {
    margin-top: 0.8em;
    display: flex;
    flex-direction: column;
    gap: 0.5em;
}

.btn {
    display: block;
    color: white;
    text-decoration: none;
    padding: 0.7em;
    border-radius: 0.6em;
    font-weight: 600;
    transition: 0.3s;
    font-size: clamp(12px, 1vw, 15px);
}

.btn.blue { background-color: #007BFF; }
.btn.green { background-color: #28a745; }
.btn.blue:hover { background-color: #0056b3; }
.btn.green:hover { background-color: #1e7e34; }

/* === Responsif untuk tablet & HP === */
@media (max-width: 768px) {
    .info-box {
        top: 1vh;
        left: 2vw;
        transform: scale(1);
        max-width: 88vw;
        padding: 0.8em 1em;
        border-radius: 0.8em;
    }

    .logo-container {
        flex-direction: column;
        gap: 0.4em;
        margin-bottom: 0.6em;
    }

    .logo-item {
        height: clamp(28px, 7vw, 40px);
    }

    .title-text {
        font-size: clamp(13px, 4vw, 17px);
        margin-bottom: 0.2em;
    }

    .subtitle-text {
        font-size: clamp(11px, 3.3vw, 13px);
        margin-bottom: 0.4em;
    }

    .desc-text {
        font-size: clamp(11px, 3.2vw, 13px);
        line-height: 1.3;
    }

    .button-container {
        gap: 0.4em;
        margin-top: 0.5em;
    }

    .btn {
        font-size: clamp(11px, 3.2vw, 13px);
        padding: 0.55em 0.7em;
        border-radius: 0.5em;
    }
}
</style>
"""
m.get_root().html.add_child(folium.Element(title_html))

# === Legenda zona kawasan responsif dan anti-scaling ===
legend_html = """
<style>
.map-legend {
    position: fixed;
    bottom: 3vh;
    left: 2vw;
    background: rgba(255, 255, 255, 0.9);
    padding: 1em 1.3em;
    border-radius: 1em;
    box-shadow: 0 0.5em 1em rgba(0,0,0,0.3);
    font-family: 'Segoe UI', Arial, sans-serif;
    font-size: clamp(12px, 1.1vw, 15px);
    z-index: 9999;
    line-height: 1.6em;
    backdrop-filter: blur(4px);
    transform: scale(1);
    transform-origin: bottom left;
}

.map-legend b {
    font-size: 1.1em;
    color: #0b3d91;
}

.legend-item {
    display: flex;
    align-items: center;
    margin: 0.2em 0;
}

.legend-color {
    width: 1.2em;
    height: 1.2em;
    border-radius: 3px;
    margin-right: 0.5em;
    border: 1px solid #555;
    flex-shrink: 0;
}

/* Responsif untuk tablet & HP */
@media (max-width: 768px) {
    .map-legend {
        bottom: 2vh;
        left: 2vw;
        transform: scale(1);
        font-size: clamp(11px, 2.5vw, 13px);
        padding: 0.8em 1em;
        max-width: 70vw;
    }
}
</style>

<div class="map-legend">
    <b>🗺️ Legenda</b>
    <div class="legend-item"><span class="legend-color" style="background:#FF0000"></span>Zona Inti</div>
    <div class="legend-item"><span class="legend-color" style="background:#96FF96"></span>Zona Pemanfaatan Terbatas</div>
    <div class="legend-item"><span class="legend-color" style="background:#828282"></span>Zona Lain</div>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# === 10. Simpan file HTML ===
Path("index.html").write_text(m.get_root().render(), encoding="utf-8")
print("✅ Peta berhasil dibuat → buka file: index.html")

✅ Peta berhasil dibuat → buka file: index.html
